<h2>Building A Chatbot </h2>
Designing and implementing an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

In [14]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key=os.getenv("GROQ_API_KEY")

In [2]:
from langchain_groq import ChatGroq
model=ChatGroq(model="openai/gpt-oss-120b",groq_api_key=groq_api_key)
model

c:\Users\Nitin Mishra\OneDrive\Desktop\AIML\Gen-AI\Langchain\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 32768, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x0000018FC1772090>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000018FC17F9C10>, model_name='openai/gpt-oss-120b', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is Nitin Mishra. Currently I am learning Langchain framework.")])

AIMessage(content='Hello Nitin! 👋 Great to meet you, and it’s exciting that you’re diving into the LangChain framework. How’s the learning journey going so far? \n\nIf you’re looking for a quick roadmap or some handy resources, here are a few suggestions to help you get the most out of LangChain:\n\n---\n\n## 1️⃣ Core Concepts to Master\n\n| Concept | What It Is | Why It Matters |\n|---------|------------|----------------|\n| **Chains** | Sequentially connect LLM calls, prompts, and other components. | Lets you build complex workflows from simple building blocks. |\n| **Agents** | LLMs that can decide which tool or chain to use next. | Enables dynamic, decision‑making pipelines (e.g., “search‑then‑summarize”). |\n| **Memory** | State‑keeping across calls (e.g., conversation history). | Makes interactions feel coherent and context‑aware. |\n| **Retrievers & Vector Stores** | Retrieve relevant documents from a knowledge base. | Powers RAG (Retrieval‑Augmented Generation) for factual accu

In [4]:
# Demonstrating a hard-coded conversation history to showcase how the model can maintain context.
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Nitin Mishra. Currently I am learning Langchain framework."),
        AIMessage(content="Hello Nitin Mishra! It's nice to meet you. \n\nAs someone learning the Langchain framework, what kind of projects are you working on these days? \n\nI'm always eager to learn more about the exciting work being done in the field of AI.\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content='Your name is **Nitin\u202fMishra**, and you’re currently **learning the LangChain framework**.', additional_kwargs={'reasoning_content': 'The user asks: "Hey What\'s my name and what do I do?" We have context: earlier user said "Hi , My name is Nitin Mishra. Currently I am learning Langchain framework." So answer: name Nitin Mishra, they are learning Langchain framework. Probably respond.'}, response_metadata={'token_usage': {'completion_tokens': 93, 'prompt_tokens': 160, 'total_tokens': 253, 'completion_time': 0.19922749, 'completion_tokens_details': {'reasoning_tokens': 61}, 'prompt_time': 0.006511732, 'prompt_tokens_details': None, 'queue_time': 0.049616918, 'total_time': 0.205739222}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_e10890e4b9', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--da948aed-0ec7-4eec-a339-5ba8a957252c-0', usage_metadata={'input_tokens': 160, 'output_tokens': 9

<h3> Message History </h3>
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [5]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [6]:
config={"configurable":{"session_id":"chat1"}}

In [7]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Nitin Mishra. Currently I am learning Langchain framework.")],
    config=config
)

In [8]:
response.content

'Hello Nitin! 👋\n\nThat’s great to hear you’re diving into LangChain. It’s a powerful framework for building LLM‑driven applications, and there’s a lot you can do with it—from simple chatbots to complex retrieval‑augmented generation pipelines.\n\nIf you’d like, I can help with:\n\n- **Getting started** – a quick “hello‑world” example and the typical project layout.  \n- **Core concepts** – chains, agents, memory, callbacks, and how they fit together.  \n- **Integrations** – connecting to vector stores (FAISS, Pinecone, Chroma, etc.), document loaders, or external APIs.  \n- **Best practices** – structuring prompts, handling errors, testing, and deployment tips.  \n- **Resources** – tutorials, official docs, community projects, and sample repos.\n\nLet me know what you’re most interested in or any specific question you have, and we can dive right in! 🚀'

In [9]:
store["chat1"] 

InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi , My name is Nitin Mishra. Currently I am learning Langchain framework.', additional_kwargs={}, response_metadata={}), AIMessage(content='Hello Nitin! 👋\n\nThat’s great to hear you’re diving into LangChain. It’s a powerful framework for building LLM‑driven applications, and there’s a lot you can do with it—from simple chatbots to complex retrieval‑augmented generation pipelines.\n\nIf you’d like, I can help with:\n\n- **Getting started** – a quick “hello‑world” example and the typical project layout.  \n- **Core concepts** – chains, agents, memory, callbacks, and how they fit together.  \n- **Integrations** – connecting to vector stores (FAISS, Pinecone, Chroma, etc.), document loaders, or external APIs.  \n- **Best practices** – structuring prompts, handling errors, testing, and deployment tips.  \n- **Resources** – tutorials, official docs, community projects, and sample repos.\n\nLet me know what you’re most interested in

Here is the step-by-step flow for a new session:

1. The Trigger:-<br>
You call invoke with a specific session_id in the config:
```t
with_message_history.invoke(
    [HumanMessage(content="Hi...")],
    config={"configurable": {"session_id": "chat1"}}
)
```

2. The Lookup (Inside RunnableWithMessageHistory):-<br>
Before sending anything to the LLM, the RunnableWithMessageHistory wrapper pauses and looks at your config. It sees session_id="chat1".

It then calls your helper function: get_session_history("chat1").

3. Creation and Storage (Inside get_session_history):-<br>
This is where the storage happens.
```t
def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        # A. Create a new empty history object
        new_history = ChatMessageHistory() 
        
        # B. Save it into the dictionary immediately
        store[session_id] = new_history 
        
    return store[session_id]
```
- Crucial Step: The moment ChatMessageHistory() is assigned to store[session_id], the dictionary now holds a reference to that history object. Any changes made to that object later will be "saved" in the dictionary because the dictionary is holding the live object.


4. The Update (Back inside RunnableWithMessageHistory)<br>
Now that the wrapper has the history object (let's call it history_obj), it does two things:

- 1. Add User Message: It takes your input ("Hi...") and calls 
```history_obj.add_user_message("Hi...").```
    <br>Result: The object inside your store dictionary now contains [HumanMessage(content="Hi...")].
- 2. Call LLM: It sends the message to the LLM and gets a response ("Hello!").<br>
- 3. Add AI Message: It takes the response and calls ```history_obj.add_ai_message("Hello!").```<br>
Result: The object inside your store dictionary now contains ```[HumanMessage(...), AIMessage(...)].```
<h6>Summary</h6>
The store isn't updated at the end; it is updated continuously.<br>

Your function puts an empty container into the store.<br>
The Runnable wrapper keeps adding messages into that container as the conversation flows.<br>
Because store holds the

container (the object), and the wrapper keeps filling that container, the store always has the latest messages.

In [10]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content='Your name is **Nitin\u202fMishra**.', additional_kwargs={'reasoning_content': 'The user asks "What\'s my name?" The user earlier said "Hi , My name is Nitin Mishra." So answer: Nitin Mishra. Should be concise.'}, response_metadata={'token_usage': {'completion_tokens': 57, 'prompt_tokens': 303, 'total_tokens': 360, 'completion_time': 0.117794174, 'completion_tokens_details': {'reasoning_tokens': 36}, 'prompt_time': 0.013201416, 'prompt_tokens_details': None, 'queue_time': 0.054650153, 'total_time': 0.13099559}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_4a19b1544c', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--f3b21bff-4d0f-4aad-ba6f-25f22249c361-0', usage_metadata={'input_tokens': 303, 'output_tokens': 57, 'total_tokens': 360, 'output_token_details': {'reasoning': 36}})

In [11]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'I don’t have any information about your name. If you’d like me to address you a certain way, just let me know!'

In [12]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is Ria")],
    config=config1
)
response.content

'Nice to meet you, Ria! How can I help you today?'

In [13]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'Your name is Ria.'

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [15]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Answer all the question to the best of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [22]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Nikhil Mishra.")]})

AIMessage(content='Hello Nikhil! 👋 Nice to meet you. How can I assist you today?', additional_kwargs={'reasoning_content': 'The user says "Hi My name is Nikhil Mishra." Probably wants a greeting. We can respond politely, ask how we can help. Also maybe ask if they\'d like to share more. Use friendly tone.'}, response_metadata={'token_usage': {'completion_tokens': 73, 'prompt_tokens': 100, 'total_tokens': 173, 'completion_time': 0.154825083, 'completion_tokens_details': {'reasoning_tokens': 45}, 'prompt_time': 0.003775973, 'prompt_tokens_details': None, 'queue_time': 0.053991677, 'total_time': 0.158601056}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_626f3fc5e0', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--3d33a8cb-e6b4-4677-8668-785f1be3b2fd-0', usage_metadata={'input_tokens': 100, 'output_tokens': 73, 'total_tokens': 173, 'output_token_details': {'reasoning': 45}})

In [23]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [25]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Nikhil Mishra.")],
    config=config
)

response.content

'Hello Nikhil! 👋 Nice to meet you. How can I assist you today?'

In [26]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Nikhil Mishra.'

In [27]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [29]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Adarsh")],"language":"Hindi"})
response.content

'नमस्ते आदर्श! आपसे मिलकर खुशी हुई। मैं आपकी कैसे मदद कर सकता हूँ?'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [30]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [31]:
config = {"configurable": {"session_id": "chat4"}}
response=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Nitin Mishra")],"language":"Hindi"},
    config=config
)
response.content

'नमस्ते, नितिन मिश्रा जी! आपका स्वागत है। मैं आपकी किस प्रकार मदद कर सकता हूँ?'

In [32]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Hindi"},
    config=config,
)
response.content

'आपका नाम नितिन मिश्रा है।'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages